# About This File

I scrape climate data from [FAO's climate information tool](https://aquastat.fao.org/climate-information-tool/) for each Nigerian state in 2022, and then combine them into one dataset for the country

The dataset contains the following features: 
- Month
- Precipitation. mm/m
- Temperature. min.℃
- Temperature. max.℃
- Temperature. Mean ℃
- Rel. Hum. %
- Sun shine. J m⁻² day⁻¹
- Wind speed (2m) m/s
- ETo. mm/m

I also used flood occurence data from [NEMA](https://data.humdata.org/dataset/nigeria-nema-flood-affected-geographical-areasnorth-east-nigeria-flood-affected-geographical-areas/resource/833fe41d-1b92-4ca8-bfa0-8b483ed81690) on flood occurences in 2022. The features i use are:
- State.
- Date of occurence.

Combining the two datasets and some research on favourable conditions for select crops i created a risk index which i use to evalutate PHL risk by region, crop, and month (season).

The index is found by assigning weights to each variable based off how they affect a crop with floods having the highest weight.

## Scraping Data From [FAO](https://aquastat.fao.org/climate-information-tool/)

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from pathlib import Path
import pandas as pd
import time

# 1. 36 states + centroid coords
states_coords = {
    'Abia': (5.5320, 7.4860),
    'FederalCapitalTerritory': (9.0765, 7.3986),
    'Adamawa': (9.3265, 12.3984),
    'Akwa Ibom': (5.0300, 7.9400),
    'Anambra': (6.2109, 7.0710),
    'Bauchi': (10.3100, 9.8400),
    'Bayelsa': (4.6400, 6.0000),
    'Benue': (7.3363, 8.8393),
    'Borno': (11.8333, 13.1500),
    'Cross River': (5.9667, 8.3333),
    'Delta': (5.5853, 5.0184),
    'Ebonyi': (5.7630, 7.9083),
    'Edo': (6.6313, 5.5516),
    'Ekiti': (7.6248, 6.8228),
    'Enugu': (6.5207, 7.4139),
    'Gombe': (10.2899, 11.1678),
    'Imo': (5.5273, 7.0258),
    'Jigawa': (12.9000, 9.5167),
    'Kaduna': (10.5105, 7.4165),
    'Kano': (12.0022, 8.5919),
    'Katsina': (12.9855, 7.6088),
    'Kebbi': (12.4500, 4.2000),
    'Kogi': (7.7986, 6.7394),
    'Kwara': (8.3983, 4.5414),
    'Lagos': (6.5244, 3.3792),
    'Nasarawa': (8.5173, 8.5167),
    'Niger': (9.6148, 6.5564),
    'Ogun': (7.1600, 4.2833),
    'Ondo': (7.2500, 5.2000),
    'Osun': (7.5000, 4.5000),
    'Oyo': (7.8731, 3.8450),
    'Plateau': (9.0000, 8.6775),
    'Rivers': (4.8500, 6.5500),
    'Sokoto': (13.0578, 5.2438),
    'Taraba': (8.0000, 10.0000),
    'Yobe': (12.0035, 11.8307),
    'Zamfara': (12.1700, 6.6700),
}

# dynamically detect which states already have CSVs so we can skip.
out_dir = Path("Datasets/AquaStat Data Each state")
exists = {
    p.stem
     .replace("_climate_2022", "")
     .replace("_", " ")
    for p in out_dir.glob("*_climate_2022.csv")
}

# 2. Selenium setup
service = Service("./chromedriver-win64/chromedriver.exe")
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 30)

base_url = "https://aquastat.fao.org/climate-information-tool/complete-climate-data"
count = 0
for state, (lat, lon) in states_coords.items():
    if state in exists:
        count += 1
        print(f"→ {state} already exists, skipping.")
        continue

    url = f"{base_url}?lat={lat}&lon={lon}&year=2022&datasource=agera5"
    driver.get(url)

    # optional short pause for JS boot
    time.sleep(15)

    # wait until table is present, with retry on timeout
    try:
        table = wait.until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, "table.data-table-2 tbody tr")))
    except TimeoutException:

        print(f"⚠ Timeout loading table for {state}")
        driver.save_screenshot(f"{state}_timeout.png")
        continue

    # now fetch rows
    rows = driver.find_elements(By.CSS_SELECTOR, "table.data-table-2 tr")

    # parse header
    headers = [th.text.strip() for th in rows[0].find_elements(By.TAG_NAME, "th")]

    data = []
    for tr in rows[1:]:
        cols = [td.text.strip().replace("\n", " ") for td in tr.find_elements(By.TAG_NAME, ("td"))]
        if cols:
            data.append(cols)

    # build DataFrame and save
    df = pd.DataFrame(data, columns=headers)
    df.to_csv(f"Datasets/AquaStat Data Each state/{state.replace(' ', '_')}_climate_2022.csv", index=False)
    count += 1
    print(f"→ Saved {state}.csv",f"{state} {count} of 36")
    time.sleep(2)   # be gentle on the server

driver.quit()

## Aggreagting Data For Each State Into One Dataset

In [ ]:
import pandas as pd
import glob
import os

# gather all per‑state CSVs
csv_files = glob.glob(r"Datasets\AquaStat Data Each state\*_climate_2022.csv")
dfs = []
for fp in csv_files:
    state = os.path.basename(fp) \
              .replace("_climate_2022.csv", "") \
              .replace("_", " ")
    df = pd.read_csv(fp)
    df["state"] = state
    dfs.append(df)

# concatenate
if dfs:
    all_states = pd.concat(dfs, ignore_index=True)

    # drop any “Total” rows under Month
    all_states = all_states[all_states["Month"] != "Total"]

    # write out
    out_path = r"Datasets/all_states_climate_2022.csv"
    all_states.to_csv(out_path, index=False)
    print(f"→ Aggregated {len(dfs)} files (minus Totals) into {out_path}")
else:
    print("⚠ No CSV files found to aggregate.")

## Adding Flood Data

In [ ]:
import pandas as pd
import calendar

# 1. load climate + state data
climate_path = r"Datasets\all_states_climate_2022.csv"
climate = pd.read_csv(climate_path)

# 2. load flood occurrences as strings, strip whitespace
flood = pd.read_csv(r"Datasets\flood_data_2022.csv", dtype=str)
flood["raw_date"] = flood["DATE OF OCCURRENCE"].fillna("").astype(str).str.strip()

# 3. drop blanks and filter valid dd/mm/YYYY dates
mask_valid = flood["raw_date"].str.match(r"^\d{1,2}/\d{1,2}/\d{4}$")
invalid = flood[~mask_valid]
if not invalid.empty:
    print("⚠ Invalid or blank dates:")
    print(invalid[["raw_date", "State"]])

flood = flood[mask_valid].copy()

# 4. parse to datetime
flood["DATE OF OCCURRENCE"] = pd.to_datetime(
    flood["raw_date"], dayfirst=True, format="%d/%m/%Y"
)

# 5. extract month abbrev (Jan, Feb, …)
flood["Month"] = flood["DATE OF OCCURRENCE"].dt.month.map(lambda m: calendar.month_abbr[m])

# 6. build set of (month, state) where flood occurred
flood_pairs = set(zip(
    flood["Month"],
    flood["State"].str.strip().str.title()
))

# 7. add flood flag to climate df
climate["flood"] = climate.apply(
    lambda r: 1 if (r["Month"], r["state"]) in flood_pairs else 0,
    axis=1
)

# 8. save augmented dataset
out_path = r"Datasets\all_states_climate_2022_with_flood.csv"
climate.to_csv(out_path, index=False)
print(f"→ Wrote augmented file with flood flag: {out_path}")

## Creating Risk Index & Exporting New Dataset

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("../Datasets/all_states_climate_2022_with_flood.csv")
df.columns = df.columns.str.replace('\n', ' ').str.strip()
df = df.rename(columns={
    'Prc. mm/m':  'Precipitation',
    'Tmp. min.': 'tmin',
    'Tmp. max.': 'tmax',
    'Tmp. Mean':'Temperature',
    'Rel. Hum. %':'Relative Humidity',
    'Sun shine':'Sunshine',
    'Wind (2m) m/s':'Wind',
    'ETo mm/m':    'Evapotranspiration',
    'State':'state'
})

# === 1. Define crop-specific weights based on climate sensitivity ===
crop_weights = {
    'Maize': {
        'Temperature': 0.25,
        'Relative Humidity': 0.25,
        'Precipitation': 0.15,
        'Wind': 0.1,
        'Evapotranspiration': 0.1
    },
    'Rice': {
        'Temperature': 0.2,
        'Relative Humidity': 0.25,
        'Precipitation': 0.2,
        'Wind': 0.1,
        'Evapotranspiration': 0.1
    },
    'Sorghum': {
        'Temperature': 0.2,
        'Relative Humidity': 0.25,
        'Precipitation': 0.2,
        'Wind': 0.1,
        'Evapotranspiration': 0.1
    },
    'Beans': {
        'Temperature': 0.25,
        'Relative Humidity': 0.25,
        'Precipitation': 0.2,
        'Wind': 0.1,
        'Evapotranspiration': 0.1
    },
    'Millet': {
        'Temperature': 0.2,
        'Relative Humidity': 0.25,
        'Precipitation': 0.15,
        'Wind': 0.1,
        'Evapotranspiration': 0.15
    }
}

# === 2. Define ideal post-harvest climate thresholds for each crop ===
thresholds = {
    'Maize': {'Temperature': 15, 'Relative Humidity': 60, 'Precipitation': 10,  'Wind': 1, 'Evapotranspiration': 5},
    'Rice': {'Temperature': 15, 'Relative Humidity': 65, 'Precipitation': 10,  'Wind': 1, 'Evapotranspiration': 5},
    'Sorghum': {'Temperature': 15, 'Relative Humidity': 60, 'Precipitation': 10,  'Wind': 1, 'Evapotranspiration': 5},
    'Beans': {'Temperature': 15, 'Relative Humidity': 60, 'Precipitation': 10,  'Wind': 1, 'Evapotranspiration': 5},
    'Millet': {'Temperature': 15, 'Relative Humidity': 60, 'Precipitation': 10,  'Wind': 1, 'Evapotranspiration': 5}
}

# === 3. Normalize variables to [0, 1] ===
vars_to_norm = set().union(*crop_weights.values())
vars_to_norm = set(var for crop in crop_weights for var in crop_weights[crop])
norm_data = {}

for var in vars_to_norm:
    arr = df[var].astype(float)
    norm_data[var] = (arr - arr.min()) / (arr.max() - arr.min())

norm_df = pd.DataFrame(norm_data)
norm_df['state'] = df['state']
norm_df['Month'] = df['Month']

# === 4. Compute distance-to-threshold scores ===
for crop in crop_weights:
    for var in crop_weights[crop]:
        if var not in df.columns:
            continue
        actual = df[var].astype(float)
        ideal_val = thresholds[crop][var]
        distance = abs(actual - ideal_val)
        max_distance = distance.max() if distance.max() > 0 else 1
        # Invert the distance to make a score between 0 (far) and 1 (ideal)
        score = 1 - (distance / max_distance)
        norm_df[f"{crop}_{var}_score"] = score

# === 5. Compute final risk index for each crop ===
for crop, weights in crop_weights.items():
    crop_risk = 0
    for var, weight in weights.items():
        score_col = f"{crop}_{var}_score"
        if score_col in norm_df.columns:
            crop_risk += norm_df[score_col] * weight
    norm_df[crop] = crop_risk
    
# === 5.1 Add flood impact ===
# Merge flood status back into norm_df
norm_df = norm_df.merge(df[['state', 'Month', 'flood']], on=['state', 'Month'], how='left')

# For rows where flood == 1, add 1 to all crop risk indices
for crop in crop_weights:
    norm_df[crop] = norm_df[crop] + norm_df['flood']

# === 6. Aggregate results by state and month ===
risk_df = (
    norm_df
    .groupby(['state', 'Month'], as_index=False)[list(crop_weights.keys())]
    .mean()
)

# === 7. Preview or export results ===
display(risk_df.tail(20))
# risk_df.to_csv("crop_climate_risk_index.csv", index=False)


In [ ]:
risk_df[risk_df['state'] =='Kano']

In [ ]:
risk_df.to_csv('../Datasets/risk_index.csv')